In [86]:
import nltk
#nltk.download('punkt')
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
import re
import pandas as pd
from nltk.corpus import stopwords
#nltk.download('stopwords')
from tqdm import tqdm
import networkx as nx
import numpy as np
import community

In [5]:
text = PlaintextCorpusReader('my_data/', '.*')

In [13]:
text.words('test_text.txt')

['My', 'name', 'is', 'August', ',', 'and', 'I', 'am', ...]

In [15]:
with open('my_data/test_text.txt') as text_file:
    raw = text_file.read()

In [21]:
tokens = nltk.word_tokenize(raw)
#tokens

In [30]:
#pattern = r"[^\w\s]"

tokens = [token.lower() for token in tokens]

In [40]:
def remove_digits(l):
    pattern = r'[0-9\p{P}]'
    l = [re.sub(pattern, '', token) for token in l]
    return l

In [41]:
    tokens = remove_digits(tokens)
tokens

error: bad escape \p at position 4

In [59]:
def clean_strings(strings):
    cleaned_strings = []
    stop_words = set(stopwords.words('english'))

    for string in strings:
        # Remove URLs
        string = re.sub(r'http\S+', '', string)

        # Remove numbers
        string = re.sub(r'[0-9]', '', string)

        # Keep only what is not punctuation
        string = re.sub(r'[^\w\s]', '', string)

        # Remove stop words and moke them
        if string not in stop_words: string = string.lower()

        # Remove empty strings
        if len(string):
            cleaned_strings.append(string)

    return cleaned_strings

In [60]:
print(clean_strings(tokens))

['my', 'name', 'is', 'august', 'and', 'i', 'am', 'writing', 'this', 'to', 'test', 'different', 'functions', 'newline', 'is', 'here', 'in', 'line', 'two', 'and', 'i', 'will', 'have', 'one', 'in', 'a', 'second', 'as', 'well', 'third', 'line', 'can', 'test', 'if', 'we', 'can', 'find', 'multiple', 'lines']


In [65]:
df_abstract = pd.read_csv('my_data/df_CSS_paper_abstract.csv')

In [84]:
cleaned_corpus = []
df_abstract['tokens'] = None
for i, row in tqdm(df_abstract.iterrows()):
    abstract = row['abstract']
    if type(abstract) == str:

        tokens = nltk.word_tokenize(row['abstract'])
        clean_tokens = clean_strings(tokens)
        cleaned_corpus.append(clean_tokens)
        df_abstract['tokens'][i] = clean_tokens
    else:
        cleaned_corpus.append(None)

35593it [01:59, 297.60it/s]
0it [00:00, ?it/s]C:\Users\Libz\AppData\Local\Temp\ipykernel_15476\4017611902.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_abstract['tokens'][i] = clean_tokens
35593it [02:17, 258.20it/s]


In [88]:
graph = nx.read_graphml('my_data/CSS_graph.graphml')

In [89]:
def louvain_communities(graph):
    # Use the louvain method to find communities
    partition = community.best_partition(graph) # community function that uses Louvain-algorithm

    # Reformat the partitioning
    communities = {}
    for node, community_id in partition.items():
        if community_id in communities:
            communities[community_id].append(node)
        else:
            communities[community_id] = [node]

    return list(communities.values())

In [98]:
communities = np.load('my_data/Louvain_communities.npy', allow_pickle=True)

In [109]:
df_abstract['tokens']


0        [the, term, linked, data, refers, to, a, set, ...
1        [the, term, linked, data, refers, to, a, set, ...
2        [the, term, linked, data, refers, to, a, set, ...
3                                                     None
4        [the, era, of, big, data, has, begun, computer...
                               ...                        
35588    [the, digitization, of, matrimonial, matchmaki...
35589    [the, misuse, of, social, media, platforms, to...
35590    [with, twitter, growing, as, a, preferred, cha...
35591    [with, twitter, growing, as, a, preferred, cha...
35592    [this, chapter, concludes, that, health, promo...
Name: tokens, Length: 35593, dtype: object

In [115]:
all_communities = []
#Test if pandas explode is more efficient
# Den her kan gøre meget bedre tror jeg, skal bare lige tænke mig lidt om
for i, row in tqdm(df_abstract.iterrows()):
    for community in communities:
        community_tokens = []
        for authorID in community:
            if authorID in row['authors']:
                tokens = row['tokens']
                community_tokens.append(tokens)
        all_communities.append(community_tokens)




616it [00:17, 35.90it/s]


KeyboardInterrupt: 

In [ ]:
#list of unique tokens
unique_tokens = list({token for sublist in all_communities for token in sublist})
token_count_dict = {token: 0 for token in unique_tokens}
for sublist in all_communities:
    for token in sublist:
        token_count_dict[token] += sublist.count(token)